In [1]:
QPATH = "Quantlet/code_description2project_description"

In [4]:
import sys
IN_COLAB = 'google.colab' in sys.modules

import os
if IN_COLAB:
  os.chdir(f'/content/drive/MyDrive/ColabNotebooks/IRTG/Encode_the_Qode/Encode-the-Qode/{QPATH}')

sys.path.append('../src')

In [50]:
import pickle
import json
import re
import sys
from IPython.display import display

from tqdm import tqdm
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
import seaborn as sns

import nltk
nltk.download('punkt')

from tqdm import tqdm
tqdm.pandas()


import importlib
importlib.reload(preprocessing_utils)
import preprocessing_utils
from preprocessing_utils import *

from sklearn.model_selection import train_test_split

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [35]:
with open('../../data/preprocessed/Quantlet/Parsed_Qs_with_code_25062023.pkl', 'rb') as file:
  df = pickle.load(file)

In [36]:
df = df[df.metainfo_file!='empty']
print(df.shape)

(4856, 6)


In [37]:
# Parse metainfo file
if 'Keywords' not in df.columns:
  meta_info = pd.DataFrame(columns=['Quantlet', 'Description', 'Keywords', 'Other'])

  meta_info[['Quantlet', 'Description', 'Keywords', 'Other']] = df.apply(
      lambda x: parse_meta(x),
      axis='columns',
      result_type='expand'
      )

  for col in meta_info.columns:
      meta_info[col] = meta_info[col].astype(str)

  df = pd.concat([df, meta_info], axis=1)

  del df['metainfo_file']
  del df['Other']
  del df['script_name']
  del df['script_name_no_ext']

In [38]:
df['multiple_scripts'] = df['code_script'].apply(lambda x: any(isinstance(i, list) for i in x))
df['code_script_joined'] = ''
df.loc[df['multiple_scripts']==True, 'code_script_joined'] = df.loc[df['multiple_scripts']==True, 'code_script'].apply(lambda x: [''.join(code_script) for code_script in x])
df.loc[df['multiple_scripts']!=True, 'code_script_joined'] = df.loc[df['multiple_scripts']!=True, 'code_script'].apply(lambda x: [''.join(x)])
df['scr_n'] = df['code_script_joined'].apply(len)
df['description_len'] = df['Description'].apply(len)
df['description_n_words'] = df['Description'].apply(lambda x: len(x.split()))
df = df.reset_index(drop=True)

In [46]:
df['main_script'] = np.nan
df['main_type_script'] = np.nan

In [75]:
def explode_code_and_lang(df):
    new_df = pd.DataFrame()

    print(f'Shape before exploding scripts: {df.shape}')

    for index, row in tqdm(df.iterrows()):
        if row['multiple_scripts']==True:
          for i, script in enumerate(row['code_script']):
              row['main_script'] = script
              row['main_type_script'] = row['type_script'][i]
              new_df = new_df.append(row)
        else:
          new_df = new_df.append(row)

    new_df['main_script'] = new_df['main_script'].fillna(new_df['code_script'])
    new_df['main_type_script'] = new_df['main_type_script'].fillna(new_df['type_script'])

    new_df = new_df.reset_index(drop=True)
    print(f'Shape after exploding scripts: {new_df.shape}')
    return new_df


In [81]:
df_long = explode_code_and_lang(df)

df_long['code_script'] = df_long['main_script'].progress_apply(lambda x: x[0] if len(x)>0 else np.nan)
df_long['type_script'] = df_long['main_type_script']

del df_long['main_type_script']
del df_long['main_script']
del df_long['code_script_joined']

Shape before exploding scripts: (4856, 13)


4856it [00:23, 208.84it/s]


Shape after exploding scripts: (6743, 13)


100%|██████████| 6743/6743 [00:00<00:00, 490882.44it/s]


In [100]:
string = """''' docstring ''' \n
some code \n
more code\n
code # comment \n
"""

In [102]:
result = re.sub(r"#+", r"<COMMENT>", string)
print(result)
result = re.sub(r'(<COMMENT>.+)\n', r'\1<COMMENT END>', result)
print(result)
result = re.sub(r'(""")(.+)\1', r'<DOCSTR START>\2<DOCSTR END>', result)
print(result)
result = re.sub(r"(''')(.+)\1", r'<DOCSTR START>\2<DOCSTR END>', result)
print(result)

''' docstring ''' 

some code 

more code

code <COMMENT> comment 


''' docstring ''' 

some code 

more code

code <COMMENT> comment <COMMENT END>

''' docstring ''' 

some code 

more code

code <COMMENT> comment <COMMENT END>

<DOCSTR START> docstring <DOCSTR END> 

some code 

more code

code <COMMENT> comment <COMMENT END>



In [98]:
result

"''' docstring '''"

In [95]:
import re
inp = '653433,78'
out = re.sub(r'(<COMMENT>.+)\n', r'\1<COMMENT END>', result)
print( out)

<COMMENT> problem: the tickers/names <COMMENT END>


In [40]:
 df_long['scr_n'] = df_long['code_script_joined'].apply(len)

In [41]:
df['Description'].nunique()

2366

In [43]:
df_long

,folder_name,code_script,type_script,Quantlet,Description,Keywords,multiple_scripts,code_script_joined,scr_n,description_len,description_n_words
0,../../data/QuantLet/Disaster/adaptive_algorithm_1,"[[close all\n, graph = 1;\n, % Open & read txt...",[m],adaptive_algorithm_1,Applies the adaptive algorithm on the model pr...,"regression, classification, stock-price, simul...",True,close all\ngraph = 1;\n% Open & read txt file ...,3772,135,22
1,../../data/QuantLet/CRIX_ETF,[[#########################################\r\...,"[r, r, r]",CRIX_ETF,Scenario analysis for an ETF on the CRIX (Haer...,"Cryptocurrency, CRIX, ETF",True,#########################################\r\n#...,9672,153,17
1,../../data/QuantLet/CRIX_ETF,[[#########################################\r\...,"[r, r, r]",CRIX_ETF,Scenario analysis for an ETF on the CRIX (Haer...,"Cryptocurrency, CRIX, ETF",True,#################################\n### Underst...,8972,153,17
1,../../data/QuantLet/CRIX_ETF,[[#########################################\r\...,"[r, r, r]",CRIX_ETF,Scenario analysis for an ETF on the CRIX (Haer...,"Cryptocurrency, CRIX, ETF",True,# problem: the tickers/names of the cryptocurr...,8613,153,17
2,../../data/QuantLet/CRIX_ETF,"[#################################\n, ### Unde...",r,CRIX_ETF,Scenario analysis for an ETF on the CRIX (Haer...,"Cryptocurrency, CRIX, ETF",False,#################################\n### Underst...,8972,153,17
...,...,...,...,...,...,...,...,...,...,...,...
4851,../../data/QuantLet/QuachSymanzikForsgren,[# This script calculates the column percentag...,r,DataExpo2013_QuachSymanzikForsgren,This repository contains supplementary materia...,"Machine Learning, Statistical Learning, Data M...",False,# This script calculates the column percentage...,6761,181,28
4852,../../data/QuantLet/QuachSymanzikForsgren,[# This script creates heatmaps of the missing...,r,DataExpo2013_QuachSymanzikForsgren,This repository contains supplementary materia...,"Machine Learning, Statistical Learning, Data M...",False,# This script creates heatmaps of the missing ...,12755,181,28
4853,../../data/QuantLet/QuachSymanzikForsgren,[# Soul of the Community: An Attempt to Assess...,r,DataExpo2013_QuachSymanzikForsgren,This repository contains supplementary materia...,"Machine Learning, Statistical Learning, Data M...",False,# Soul of the Community: An Attempt to Assess ...,1620,181,28
4854,../../data/QuantLet/QuachSymanzikForsgren,[# This script runs random forests only on Gar...,r,DataExpo2013_QuachSymanzikForsgren,This repository contains supplementary materia...,"Machine Learning, Statistical Learning, Data M...",False,# This script runs random forests only on Gary...,8832,181,28
